In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [8]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas.core import datetools

from sklearn.preprocessing import LabelEncoder

import numpy as np

import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm


In [4]:
# df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HealthMart_Performance_Data_clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c')
# df_raw = pd.read_csv('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HealthMart_Performance_Data_clean.csv')
df_raw = pd.read_excel('c:/users/elkzze1/OneDrive - McKesson Corporation/training/ds201_capstone/data/HM_Master_Data_Analytics_Clean.xlsx', sheet_name='Health_Mart_Store_Progam_List_c')
df_raw.head()

,Account,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,...,Capture_Rate,Specialty_Spend,Specialty_Product_Purchases,Generic,Brand,Generic_Ratio,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89
0,10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_raw.columns

Index(['Account', 'Nabp_Ncpdp', 'Npi_Num', 'Dea_Num', 'Target', 'Store_Name',
       'Address-Delivery', 'City-Delivery', 'ST', 'Zip', 'Owner_Mgr_Full_Name',
       'Owner_Email', 'Phone_Number', 'Hm_Start_Dt', 'Channel_Type_Cd',
       'Cust_Type_Desc', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded',
       'Buying_Group', 'Chain_Name', 'Sub_Group_Name', 'Region_Name',
       'Region_Director', 'Pharm_D', 'Vpgm_Name', 'Vps_Name', 'Dsm_Name',
       'RSM_Name', 'Salesperson', 'Salesperson_Group', 'AH_Program',
       'Auto_Ship_Ind', 'DLC_Program', 'FEM_Program', 'HM_Circular_Program',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuI

In [6]:
df_raw['ST'].value_counts()

CA    436
FL    353
TX    344
AL    328
MI    316
NY    256
GA    205
KY    160
IL    155
OK    146
OH    129
TN    116
MO    111
NJ    110
PA    108
WI     97
MS     94
SC     93
MD     93
KS     89
AR     80
NE     77
IA     73
CO     69
NC     69
LA     59
CT     58
WA     52
UT     51
WV     50
MN     43
VA     41
IN     41
SD     39
HI     37
MA     37
ND     35
OR     27
MT     27
AZ     22
ID     21
NV     18
NM     17
DE     14
DC     11
RI     10
WY      8
ME      7
AK      6
VT      5
NH      3
Name: ST, dtype: int64

In [13]:
# if you want to operate on multiple columns, put them in a list like so:
# dollar_cols = ['Interior Decor / Signage Available Funds', 'LMS Available Matching Funds', 'Tot Sls Amt', 'Brand Rx Sls Amt',
#                'Gnrc_Rx_Sls_Amt','Os Gnrc Rx Sls Amt','Otc Sls Amt']
X_cols = ['Internal_Decor', 'Open_Smart', 'PIP_Program', 'PQS_Program', 'SCS_Program', 'Specialty_Solutions', 'Vaccine_Items',
          'Vaccine_Starter', 'YPO']
y_n_cols = ['340B_Active', 'LMS_Enrolled', 'LMS_Used_Funds_Most_Recent_3_Months', 'LMS_Used_Funds_Previous_3_Months']
yes_cols = ['Auto_Ship_Ind', 'Spar_Coverage_Ind']
cat_cols = ['DLC_Program', 'FEM_Program', 'HM_Circular_Program']
# pass them to df.replace(), specifying each char and it's replacement:
# df_raw[dollar_cols] = df_raw[dollar_cols].replace({'\$': '', ',': '',' -   ': '','-': '' }, regex=True)
# df_clean[dollar_cols] = df_raw[dollar_cols].fillna(0)
# df_clean.fillna(0, inplace=True)
# df_raw['YPO'] = df_raw['YPO'].apply(lambda x: 1 if x == 'X' else 0).astype(int)
# df_raw['AH Program'] = df_raw['AH Program'].apply(lambda x: 1 if x == 'X' else 0).astype(int)
# df_clean['340B Active'] = df_clean['340B Active'].map({'N': 0, 'Y' : 1})
# df_raw['340B Active'] = df_raw['340B Active'].apply(lambda x: 1 if x == 'Y' else 0).astype(bool).astype(int)
# df_raw = fill_missing_values(df_raw)
# df_clean = replace_chars(df_raw, dollar_cols)
df_clean[X_cols] = df_raw[X_cols].applymap(lambda x: 1 if x == 'X' else 0)
df_clean[y_n_cols] = df_raw[y_n_cols].applymap(lambda x: 1 if x == 'Y' else 0)
df_clean[yes_cols] = df_raw[yes_cols].applymap(lambda x: 1 if x == 'Yes' else 0)
df_clean['3rd_Party_Vendor'] = df_raw['3rd_Party_Vendor'].apply(lambda x: 'UNK' if pd.isnull(x) else x).astype(str)
df_clean['Hospital_Associated'] = df_raw['Hospital_Associated'].apply(lambda x: 'UNK' if pd.isnull(x) else x).astype(str)
# df_raw[X_cols] = df_raw[X_cols].apply(clean_X)
# df_raw['YPO'] = df_raw['YPO'].apply(clean_X)
df_clean['DLC_Program'] = df_raw['DLC_Program'].apply(lambda x: 'DLC_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean['FEM_Program'] = df_raw['FEM_Program'].apply(lambda x: 'FEM_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))
df_clean['HM_Circular_Program'] = df_raw['HM_Circular_Program'].apply(lambda x: 'HM_Circular_Program' if x == 'X' else ('Unknown' if pd.isnull(x) else x))


In [96]:
# df_clean['HM Circular Program'].unique
# df_clean['DLC Program'].value_counts()
# df_clean['DLC Program'].isnull().values.any()
# df_raw['3rd Party Vendor'] = df_raw['3rd Party Vendor'].astype(str)
# tmp = df_raw['3rd Party Vendor'].fillna(''); isEmpty = tmp==''
# tmp

In [14]:
# df_raw['3rd Party Vendor'].dtype
df_clean.head(5)

,Account,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,...,Specialty_Spend,Specialty_Product_Purchases,Generic,Brand,Generic_Ratio,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,ST_encoded
0,10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
1,17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
2,19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
3,309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
4,652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37


In [15]:
# df_clean = df_raw[['Account','Store Name','AddressDelivery','CityDelivery','ST','Zip','Region','DC Name','PSAO Expanded',
#                    'Buying Group','AH Program','Auto Ship Ind','DLC Program','FEM Program','HM Circular Program',
#                    'Internal Decor','Interior Decor / Signage Available Funds','LMS Enrolled','LMS Available Matching Funds',
#                    'LMS Used Funds Most Recent 3 Months','LMS Used Funds Previous 3 Months','Open Smart',
#                    'MRA Program Type  Active AH','OTC Frontend Size','PIP Program','Pog Code Name', 'PQS Enrolled',
#                    'Last EQuIPP Login','SCS Program','Specialty Solutions','Vaccine Items','Vaccine Starter',
#                    'Vaccine Items Count','Vaccine Items Sls Amt','YPO','Tot Sls Amt','Brand Rx Sls Amt','Gnrc Rx Sls Amt',
#                    'Os Gnrc Rx Sls Amt','Otc Sls Amt','3rd Party Vendor','Hospital Associated','340B Active']]
# df_clean.head()

In [63]:
# df_clean['3rd Party Vendor'].value_counts()

In [64]:
# df_clean['Interior Decor / Signage Available Funds'].value_counts

In [16]:
df_clean.columns

Index(['Account', 'Nabp_Ncpdp', 'Npi_Num', 'Dea_Num', 'Target', 'Store_Name',
       'Address-Delivery', 'City-Delivery', 'ST', 'Zip', 'Owner_Mgr_Full_Name',
       'Owner_Email', 'Phone_Number', 'Hm_Start_Dt', 'Channel_Type_Cd',
       'Cust_Type_Desc', 'Bus_Type_Desc', 'Region', 'DC_Name', 'PSAO_Expanded',
       'Buying_Group', 'Chain_Name', 'Sub_Group_Name', 'Region_Name',
       'Region_Director', 'Pharm_D', 'Vpgm_Name', 'Vps_Name', 'Dsm_Name',
       'RSM_Name', 'Salesperson', 'Salesperson_Group', 'AH_Program',
       'Auto_Ship_Ind', 'DLC_Program', 'FEM_Program', 'HM_Circular_Program',
       'Internal_Decor', 'Interior_Decor_Signage_Available_Funds',
       'LMS_Enrolled', 'LMS_Available_Matching_Funds',
       'LMS_Used_Funds_Most_Recent_3_Months',
       'LMS_Used_Funds_Previous_3_Months', 'Open_Smart',
       'MRA_Program_Type_-_Active_AH', 'Msa_Dma', 'OTC_Front-end_Size',
       'PIP_Program', 'Pog_Code_Name', 'KYN_Training', 'PQS_Enrolled',
       'PQS_Program', 'Last_EQuI

In [ ]:
le = LabelEncoder()
df_clean['ST_encoded'] = le.fit_transform(df_clean['ST'])
df_clean.head(5)

In [23]:
label_encoder=LabelEncoder()
# X['DLC Program'] = labelencoder_X.fit_transform(X['DLC Program'])
# df["ModelNumeric"] = df["Model"].astype("category", ordered = True).cat.codes # Thank you B.Turnwald
df_clean['DLC_Program_encoded'] = label_encoder.fit_transform(df_clean['DLC_Program'])

In [20]:
df_clean['DLC_Program'].unique()

array(['DLC_Program', 'Unknown', 'Limited Front-end', 'No Front-end',
       'Removed', 'Refusal'], dtype=object)

In [21]:
# print(df_raw.loc[df_raw['340B Active'] == 1])
# df_is = df_clean.loc[df_clean['340B Active'] == 1]
# df_is.head()

In [24]:
df_clean.head()

,Account,Nabp_Ncpdp,Npi_Num,Dea_Num,Target,Store_Name,Address-Delivery,City-Delivery,ST,Zip,...,Generic,Brand,Generic_Ratio,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,ST_encoded,DLC_encoded,DLC_Program_encoded
0,10491,3800047,1205901485,AM1635474,10491.0,POSTAL PHARMACY,47809 HIGHWAY 58,OAKRIDGE,OR,97463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,0,0
1,17475,3802421,1912059569,AH1621336,17475.0,HOWARDS DRUGS,101 NORTH F STREET,LAKEVIEW,OR,97630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,0,0
2,19901,3804425,1114456563,AK1605887,19901.0,HARVARD AVENUE DRUGS,1175 HARVARD AVE,ROSEBURG,OR,97471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,0,0
3,309745,3804324,1437294808,FS5689964,309745.0,REEDSPORT PHARMACY,1409 HWY 101,REEDSPORT,OR,97467,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,5,5
4,652229,3844366,1275808743,FJ3131290,652229.0,PHILOMATH FAMILY PHARMACY,1640 MAIN STREET,PHILOMATH,OR,97370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,1,1
